# Session 3: Holistic Benchmarks

In this coding example, we will learn how to estimate the quantum volume of a backend using Qiskit's quantum volume module.  The method is based on the paper "Validating quantum computers using randomized model circuits" (https://arxiv.org/abs/1811.12926). For more details, see [this chapter](https://qiskit.org/textbook/ch-quantum-hardware/measuring-quantum-volume.html) in the Qiskit textbook, or this [Qiskit Medium blog post](https://medium.com/qiskit/what-is-quantum-volume-anyway-a4dff801c36f).



In [ ]:
#Import general libraries (needed for functions)
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

#Import Qiskit classes classes
import qiskit
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error

#Import the qv function.
import qiskit.ignis.verification.quantum_volume as qv

Let's look at the quantum volume of various backends.

In [ ]:
from qiskit import IBMQ
IBMQ.load_account()

In [ ]:
hub = 'ibm-q'
group = 'open'
project = 'main'

provider = IBMQ.get_provider(hub, group, project)

provider.backends()

In [ ]:
for be in ['ibmqx2', 'ibmq_16_melbourne', 'ibmq_vigo', 'ibmq_ourense', 'ibmq_valencia', 'ibmq_armonk', 'ibmq_athens', 'ibmq_santiago']:
    backend = provider.get_backend(be)
    props = backend.configuration()
    print('Backend : {0}'.format(be))
    print('Number of qubits: {0}'.format(props.n_qubits))
    print('Quantum volume: {0}'.format(props.to_dict()['quantum_volume']))
    print('------------')

## Generate a quantum volume experiment design

The experiment design for a quantum volume experiment consists of sets of circuits acting on a given number of qubits. Given a circuit width $w$, an ensemble of circuits is generated, where each circuit in the ensemble consists of a random realization of a quantum volume circuit.

Here, we will look at quantum volume circuits that act on up to 5 qubits. Note: Because quantum volume circuits include 2-qubit gates, the minimum number of qubits (width) is 2.

In [ ]:
# Generate a list of qubits of varying lengths.
# The length of each sub-list is the width.
qubit_list = [list(range(width)) for width in range(2, 6)]

# Set the number of trials (random realizations) for each width
# to be 50
numTrials = 100

In [ ]:
qubit_list

In [ ]:
qv_circs, qv_circs_nomeas = qv.qv_circuits(qubit_list, ntrials=numTrials)

### Look at the circuits

The ensemble is indexed as (random realization, number of qubits).

In [ ]:
# Check the shape of the returned object.
np.shape(qv_circs)

In [ ]:
# Draw the first circuit in the list
qv_circs[0][0].draw(output='mpl')

In [ ]:
# Draw the first circuit in the list, but without the terminating measurements
qv_circs_nomeas[0][0].draw(output='mpl')

Let's take a circuit and pass it through the transpiler to unroll it.

In [ ]:
# Transpile the first circuit to the u1,u2,u3,cx basis set.
transpiled_circuit = qiskit.compiler.transpile(qv_circs[0][0], basis_gates=['u1', 'u2', 'u3','cx'])

As an example, we print the circuit corresponding to the first QV sequence. Note that the ideal circuits are run on the first n qubits (where n is the number of qubits in the subset).

In [ ]:
# Draw the transpiled circuit
transpiled_circuit.draw(output='mpl')

## Simulate the ideal circuits

Estimating the quantum volume method requires that we know the ideal output for each circuit, so use Qiskit's to get the ideal result.

In [ ]:
backend = qiskit.Aer.get_backend('statevector_simulator')
ideal_results = []
for trial in range(numTrials):
    if trial%10 == 0: print('Simulating trial %d'%trial)
    ideal_results.append(qiskit.execute(qv_circs_nomeas[trial], backend=backend, optimization_level=0).result())

## Simulate running on hardware

We define a noise model for the simulator, using the noise properties of a given backend.

In [ ]:
# Pick a 5-qubit backend; e.g., valencia, athens, santiago
be = 'ibmq_valencia'
device_backend = provider.get_backend(be)
noise_model = NoiseModel.from_backend(device_backend)
print(noise_model)

In [ ]:
basis_gates = noise_model.basis_gates
coupling_map = device_backend.configuration().coupling_map

In [ ]:
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx']
shots = 10**4
exp_results = []
for trial in range(numTrials):
    if trial%10 == 0: print('Running trial %d'%trial)
    exp_results.append(qiskit.execute(qv_circs[trial], basis_gates=basis_gates,\
                                      backend=backend, noise_model=noise_model, coupling_map=coupling_map,\
                                      backend_options={'max_parallel_experiments': 0}, optimization_level=3).result())

## Analyze the data

Analyzing the data means determining, for each circuit width, the probability the quantum computer products _heavy outputs_, which are bitstrings whose (ideal) probability of occuring is higher than the median ideal probability.

In [ ]:
# Instantiate a QV fitter object
qv_fitter = qv.QVFitter(qubit_lists=qubit_list)

In [ ]:
# Add ideal statevectors
qv_fitter.add_statevectors(ideal_results)

# Add experimental data
qv_fitter.add_data(exp_results)

Now, we'll make a plot of the data.

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.gca()

# Plot the data
qv_fitter.plot_qv_data(ax=ax, show_plt=False)

# Add title and label
ax.set_title('Simulating a Quantum Volume experiment \n on {0} ({1} trials)'.format(be, numTrials), fontsize=18)

plt.show()

While the plot gives us some sense as to whether heavy outputs are in fact being produced, we should take a look at more in-depth statstistics to see whether circuits of a given width pass the heavy output generation test. For each depth list if the depth was successful or not and with what confidence interval. For a depth to be successful the confidence interval must be > 97.5%.

In [ ]:
qv_success_list = qv_fitter.qv_success()
qv_list = qv_fitter.ydata
for qidx, qubits in enumerate(qubit_list):
    hog_prob = np.round(qv_list[0][qidx], 5)
    confidence = np.round(qv_success_list[qidx][1], 5)
    is_successful = qv_success_list[qidx][0]
    numQubits = len(qubits)
    print('At width/depth {0}, heavy output generation probability is {1} with confidence {2}.'.format(numQubits, hog_prob, confidence))
    if is_successful:
        print('Quantum volume test passed for QV{0}!'.format(2**numQubits))
    else:
        print('Quantum volume test failed!')